<a href="https://colab.research.google.com/github/norrisandrew/Solving_PDEs/blob/main/MaterialsProject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, scipy as sp, matplotlib.pyplot as plt
import xlrd, csv      # for read to .xls, read/write to csv
from sklearn.model_selection import train_test_split    # machine learning library
from sklearn import datasets
import sklearn.linear_model
import sklearn.neighbors

In [ ]:
!pip install pymatgen==2022.0.17 # <-------------- EXTREMELY IMPORTANT

In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Composition
import re
import pprint

In [ ]:
# Make sure that you have the Materials API key. Put the key in the call to
# MPRester if needed, e.g, MPRester("MY_API_KEY")
mpr = MPRester("iR6z9LivAfSmJ0Zdd")

In [ ]:
elastic_data = mpr.query({"elasticity": {"$exists": True}},              # all the elastic data with materials ids, see http://matgenb.materialsvirtuallab.org/2017/03/02/Getting-data-from-Materials-Project.html
                         properties=["task_id", "pretty_formula", "elasticity", "elastic_tensor","structure"])

In [ ]:
print(len(elastic_data))
pprint.pprint(elastic_data[1])


In [ ]:
pprint.pprint(elastic_data[0]["pretty_formula"])
pprint.pprint(elastic_data[0]["elasticity"]["elastic_tensor"])
struct = (elastic_data[0]["structure"])
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
finder = SpacegroupAnalyzer(struct)
finder.get_crystal_system() 
finder.get_lattice_type() 
finder.get_space_group_number() 

In [ ]:
import pandas as pd
r_array = []
l1_array = []
l2_array = []
l3_array = []
l4_array = []
l5_array = []
l6_array = []
length_elastic_data = len(elastic_data)

for i in range(length_elastic_data):
        r_array.append(elastic_data[i]["pretty_formula"])
        l1_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][0])
        l2_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][1])
        l3_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][2])
        l4_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][3])
        l5_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][4])
        l6_array.append(elastic_data[i]["elasticity"]["elastic_tensor"][5])

Data = {'Formula': r_array, 'Elastic moduli row 1': l1_array, 'Row 2': l2_array, 'Row 3': l3_array, 'Row 4': l4_array, 'Row 5': l5_array, 'Row 6': l6_array}

DATA = pd.DataFrame(Data)

In [ ]:
DATA.head(3)

# Now compute the cubic elastic moduli for each of the 13k or so materials in the elastic data.

In [ ]:
form_list = [] # formula
k_list  = []   # kappa
m1_list = []   # shear modulus 1
m2_list = []   # shear modulus 2
m_list  = []   # shear modulus of iso
dc_list = []   # relative distance from cubic approximation
di_list = []   # relative distance from isotropic approximation

for i in range(length_elastic_data):
  C = np.array( elastic_data[i]["elasticity"]["elastic_tensor"] )      # converts list to array
  CdotH  = C[0,0]+C[1,1]+C[2,2]
  Cdot3J = CdotH + 2*(C[0,1]+C[1,2]+C[0,2])
  CdotI  = 2*np.trace(C) - CdotH 
  kap = Cdot3J/9                 # kappa or bulk modulus
  mu1 = (CdotI - CdotH)/6        # mu1 which is the same as mu in DOI: 10.1121/1.2173525
  mu2 = CdotH/4 - Cdot3J/12      # mu2 which is the same as eta 
  mu = (3*mu1+2*mu2)/5           #  isotropic mu

  C11 = kap + mu2*4/3   # cubic C11
  C12 = kap - mu2*2/3   # cubic C12
  C66 = mu1             # cubic C66
  C_cub = np.array( [ [C11,C12,C12,0,0,0], [C12,C11,C12,0,0,0], [C12,C12,C11,0,0,0], [0,0,0,C66,0,0], [0,0,0,0,C66,0], [0,0,0,0,0,C66] ])   # the cubic approximation of C
  C11 = kap + mu*4/3    # isotropic C11
  C12 = kap - mu*2/3    # isotropic C12
  C66 = mu              # isotropic C66
  C_iso = np.array( [ [C11,C12,C12,0,0,0], [C12,C11,C12,0,0,0], [C12,C12,C11,0,0,0], [0,0,0,C66,0,0], [0,0,0,0,C66,0], [0,0,0,0,0,C66] ])   # the isotopic approximation of C

  C_mag    = np.linalg.norm(C,2)     #   norm(C,2) is the same as norm(C)
  dist_cub = np.linalg.norm(C - C_cub)/C_mag    #  the relative distance of C_cub from C, in an L2 sense
  dist_iso = np.linalg.norm(C - C_iso)/C_mag    #  the relative distance of C_iso from C, in an L2 sense
  
  if dist_cub < 1:
    form_list.append(elastic_data[i]["pretty_formula"])
    k_list.append( kap )
    m1_list.append( mu1 )
    m2_list.append( mu2 )
    dc_list.append( dist_cub )
    m_list.append( mu )
    di_list.append( dist_iso )
                   
Cubic_data = {'Formula': form_list, 'kappa': k_list, 'mu1': m1_list, 'mu2': m2_list,  'mu': m_list, 'rel dist to cubic': dc_list, 'rel dist to isotropic': di_list}

dc_list_sort = np.sort(dc_list)            # sort the distances
di_list_sort = np.sort(di_list)            # sort the distances

plt.plot( dc_list_sort )
plt.ylabel('Relative distance to cubic')
plt.grid(True)
plt.show()

print(C)
if dist_cub < 1:
  print(dist_cub)

#print(di_list_sort[60:100] )

#print(type(C))

#d_len = len(di_list)

#print(d_len, di_list_sort[d_len-1])


The plot shows the "distance to cubic" for all 13000 materials.  A zero distance means it is actucally a cubic crystal.  A non-zero value means the crystal structure is more complicated.  A small distance means it is "almost" cubic, i.e. its properties canbe approximated by a cubic materia with the three elastic moduli: kap, mu1 and mu2